In [112]:
from py2neo import Graph, Node, Relationship
import json
import Levenshtein

In [113]:
g = Graph(password="1234")

In [114]:
with open('google.json') as f:
    google_data = json.load(f)
with open('apple.json') as f:
    apple_data = json.load(f)
with open('microsoft.json') as f:
    microsoft_data = json.load(f)
with open('fitbit.json') as f:
    fitbit_data = json.load(f)
with open('facebook.json') as f:
    facebook_data = json.load(f)
with open('Motor.json') as f:
    motorola_data = json.load(f)
with open('Youtube.json') as f:
    youtube_data = json.load(f)

In [115]:
google_node = Node("Company", name = google_data[0]['Title'], website=google_data[0]['Website'])
apple_node = Node("Company", name = apple_data[0]['Title'], website=apple_data[0]['Website'])
microsoft_node = Node("Company", name = microsoft_data[0]['Title'], website=microsoft_data[0]['Website'])
fitbit_node = Node("Company", name = fitbit_data[0]['Title'], website=fitbit_data[0]['Website'])
facebook_node = Node("Company", name = facebook_data[0]['Title'], website=facebook_data[0]['Website'])
motorola_node = Node("Company", name = motorola_data[0]['Title'], website=motorola_data[0]['Website'])
youtube_node = Node("Company", name = youtube_data[0]['Title'], website=youtube_data[0]['Website'])

In [116]:
COMPETES_WITH = Relationship.type("COMPETES_WITH")
ACQUIRED_BY = Relationship.type("ACQUIRED_BY")
g.merge(COMPETES_WITH(apple_node, google_node), "Company", "name")
g.merge(COMPETES_WITH(microsoft_node, google_node), "Company", "name")
g.merge(COMPETES_WITH(facebook_node, google_node), "Company", "name")
g.merge(ACQUIRED_BY(fitbit_node, google_node), "Company", "name")
g.merge(ACQUIRED_BY(motorola_node, google_node), "Company", "name")
g.merge(ACQUIRED_BY(youtube_node, google_node), "Company", "name")

In [117]:
for i in facebook_data[0]['Founders'].split(','):
    g.create(Relationship(facebook_node, "FOUNDED_BY", Node("Person", name=i)))
for i in microsoft_data[0]['Founders'].split(','):
    g.create(Relationship(microsoft_node, "FOUNDED_BY", Node("Person", name=i)))
for i in google_data[0]['Founders'].split(','):
    g.create(Relationship(google_node, "FOUNDED_BY", Node("Person", name=i)))
for i in apple_data[0]['Founders'].split(','):
    g.create(Relationship(apple_node, "FOUNDED_BY", Node("Person", name=i)))
for i in fitbit_data[0]['Founders'].split(','):
    g.create(Relationship(fitbit_node, "FOUNDED_BY", Node("Person", name=i)))

In [118]:
for i in facebook_data[0]['Industry'].split(','):
    temp = Node("Industry", name=i.lstrip())
    g.merge(temp, "Industry", "name")
    temp1 = Relationship(facebook_node, "BUSINESS", temp)
    g.merge(temp1)     

for i in microsoft_data[0]['Industry'].split(','):
    temp = Node("Industry", name=i.lstrip())
    g.merge(temp, "Industry", "name")
    temp1 = Relationship(microsoft_node, "BUSINESS", temp)
    g.merge(temp1)     

for i in google_data[0]['Industry'].split(','):
    temp = Node("Industry", name=i.lstrip())
    g.merge(temp, "Industry", "name")
    temp1 = Relationship(google_node, "BUSINESS", temp)
    g.merge(temp1)     

for i in apple_data[0]['Industry'].split(','):
    temp = Node("Industry", name=i.lstrip())
    g.merge(temp, "Industry", "name")
    temp1 = Relationship(apple_node, "BUSINESS", temp)
    g.merge(temp1) 

for i in youtube_data[0]['Industry'].split(','):
    temp = Node("Industry", name=i.lstrip())
    g.merge(temp, "Industry", "name")
    temp1 = Relationship(youtube_node, "BUSINESS", temp)
    g.merge(temp1)
    
for i in fitbit_data[0]['Industry'].split(','):
    temp = Node("Industry", name=i.lstrip())
    g.merge(temp, "Industry", "name")
    temp1 = Relationship(fitbit_node, "BUSINESS", temp)
    g.merge(temp1)
    
for i in motorola_data[0]['Industry'].split(','):
    temp = Node("Industry", name=i.lstrip())
    g.merge(temp, "Industry", "name")
    temp1 = Relationship(motorola_node, "BUSINESS", temp)
    g.merge(temp1) 

In [119]:
# get company name 
company_name_list = g.run("MATCH (c:Company) RETURN c.name as name").data()

In [120]:
# load naics related data
f = open("./2012_NAICS_Structure.csv", "r")
content = f.read().split("\n")
with open('naics.json') as f:
    naics_data = json.load(f)

In [121]:
#get naics code book as a dictionary
naics_code_book = {}
for line in content:
    temp_code = line.split(",")[0]
    temp_desc = line.split(",")[1]
    naics_code_book[temp_code] = temp_desc  

In [122]:
BELONGS_TO = Relationship.type("BELONGS_TO")

In [123]:


def getKey(item):
    return item[0]

for obj in naics_data:
    name = obj['name'].split(" ")[0]
    l = [(Levenshtein.distance(item['name'].split(" ")[0],name),item) for item in company_name_list]
    l = sorted(l, key=getKey)
    company_node = g.nodes.match('Company', name=l[0][1]['name']).first()
    
    prev = company_node
    cur_code = obj['code']
    while len(cur_code) > 0:
        if cur_code in naics_code_book:
            info = naics_code_book[cur_code]
            info = info.replace("\"","").strip()
            if info[-1] == "T":
                info = info[0:-1]
            temp = Node("NAICS_CODE", code=cur_code, description=info)
            g.merge(temp, "NAICS_CODE", "code")
            temp_r = BELONGS_TO(prev, temp)
            g.create(temp_r)
            prev = temp
        cur_code = cur_code[0:-1]